# Extractor

In this notebook, we make a clean dataset of all information we can extract on farm factories

We cross two tables
- One that gives us the (limited) table of all farms. https://www.georisques.gouv.fr/risques/installations/donnees?page=1&nomenclature=2101,2102,2111
- The other that gives every INSEE code for a given zip code.

This allows us to extend the first table by making requests on the API for each facility

In [1]:
import requests
import json
import pandas as pd
import numpy as np

## Load INSEE code <-> zip code

In [2]:
code_insee = pd.read_csv("code_insee.csv", usecols=["Code_commune_INSEE", "Code_postal"])
code_insee = code_insee.drop_duplicates(subset="Code_commune_INSEE")
code_insee = code_insee.dropna()

In [3]:
code_insee

,Code_commune_INSEE,Code_postal
0,1004,1500
1,1025,1380
2,1029,1270
3,1033,1200
6,1043,1700
...,...,...
39147,98823,98823
39168,98733,98711
39179,98751,98783
39189,98819,98821


## Load farms zip codes

Get every zip codes where there are farms

In [4]:
farms_zip_codes = pd.read_csv("farms_2023-02-01.csv", usecols=["Code postal"])
farms_zip_codes = farms_zip_codes.drop_duplicates()
farms_zip_codes = farms_zip_codes.dropna()

In [5]:
farms_zip_codes = farms_zip_codes.drop_duplicates().dropna()

In [6]:
farms_zip_codes

,Code postal
0,50410.0
1,50850.0
2,62810.0
3,50610.0
4,50520.0
...,...
11552,63880.0
11554,73530.0
11569,42580.0
11594,2510.0


## Get all zip codes where there are farms

In [7]:
farms_insee_codes = farms_zip_codes.merge(
    code_insee, left_on="Code postal", right_on="Code_postal")
farms_insee_codes = farms_insee_codes.drop("Code postal", axis=1)

In [8]:
farms_insee_codes

,Code_commune_INSEE,Code_postal
0,50357,50410
1,50393,50410
2,50637,50410
3,50295,50410
4,50338,50410
...,...,...
18132,2386,2510
18133,87076,87370
18134,87083,87370
18135,87013,87370


## Pull every facilities from api calls

Hypothesis: "Raison Sociale" is unique for each facility in France

In [9]:
def get_facilities_at(code_insee: int) -> list:
    request = "https://www.georisques.gouv.fr/api/v1/installations_classees?\
        page=1&page_size=100&rayon=10000&code_insee={}".format(code_insee)

    for attempt in range(10):
        try:
            response = requests.get(request).text
        except requests.exceptions.ConnectTimeout:
            print("ConnectTimeout: {} at attempt {}".format(request, attempt))
        else:
            break
    else:
        print("Warning: all attempt failed with request to {}".format(request))
        return []

    response_json = json.loads(response)

    return response_json["data"]

In [10]:
def pull_facilities(code_insee_list: list, unique_facilities: dict = {}) -> dict[str, object]:
    """
    Make an API call for each insee code.

    Returns the modified dictionary of unique facilities around INSEE code.
    the key for each facility is the "raison sociale"
    """

    for i in range(len(code_insee_list)):
        code_insee = code_insee_list[i]

        facilities_around_insee_loc = get_facilities_at(code_insee)

        for facility in facilities_around_insee_loc:
            raison_sociale: str = facility["raisonSociale"]

            if len(raison_sociale) == 0:
                print("Warning: empty name for raisonSociale")
                continue

            facility.pop("raisonSociale")
            unique_facilities[raison_sociale] = facility
        
        print("\rpull_facility() progress: {}/{}".format(i + 1, len(code_insee_list)), end='\t\t')
    print()
    
    return unique_facilities

### Parallelize to accelerate pulling the data

We divide the list of INSEE codes in N slices.

We then create N threads running in parallel and making requests to the API.

The result is written in a shared dictionary of facilities


Too much threads may result in ConnectTimeout exceptions. The API blocks the request if it receives too many requests.

In [11]:
import threading

def run_threads(facilities: dict, num_threads:int = 16):
    threads = list()

    for code_insee_slice in np.array_split(farms_insee_codes.Code_commune_INSEE, num_threads):
        x = threading.Thread(target=pull_facilities, args=(list(code_insee_slice), facilities,))
        threads.append(x)
        x.start()

    for index, thread in enumerate(threads):
        thread.join()
        print("End of a thread {}".format(index))

In [12]:
# the final list of facilites
# each thread are modifying it. But since it is a dictionnary there are no interferences
facilities = {}

# uncomment this to make threaded API calls and pull the facilities data
#run_threads(facilities)

# or uncomment this to load a cache file
with open("facilities.json", "r") as f:
    facilities = json.load(f)

## Filter

### Remove facilities that are not farms

In [13]:
def is_farm(facility: object) -> bool:
    return facility["bovins"] or facility["porcs"] or facility["volailles"]

farms = {k: v for k, v in facilities.items() if is_farm(v)}

### Make Rubric table

In [14]:
rubrics = dict()

# construct a dictionary of rubrics like this: {rubric_id: rubric_name, ...}
for farm_name, farm in farms.items():
    farm_rubrics = farm["rubriques"]
    for rubric in farm_rubrics:
        rubrics[rubric["numeroRubrique"]] = rubric["nature"]

### Make stock table

In [15]:
stocks = list()

# construct a list of stocks
for farm_name, farm in farms.items():
    farm_rubrics = farm["rubriques"]
    for rubric in farm_rubrics:
        stock = {
            "raison_sociale": farm_name,
            "rubric_id": rubric["numeroRubrique"],
            "quantite": rubric["quantiteTotale"] if "quantiteTotale" in rubric else "",
            "unite": rubric["unite"] if "unite" in rubric else "",
            }

        stocks.append(stock)

### Remove unused attributes for our project

In [16]:
def formal_notice(farm: object) -> bool:
    """
    return True if the farm has at least one document mentionning "mise en demeure"
    """

    for document in farm["documentsHorsInspection"]:
        if "mise en demeure" in document["typeFichier"].lower():
            return True
    
    return False

def alter_farm(farm: object) -> object:
    farm.pop("coordonneeXAIOT", None)
    farm.pop("coordonneeYAIOT", None)
    farm.pop("codeAIOT", None)
    farm.pop("systemeCoordonneesAIOT", None)
    farm.pop("serviceAIOT", None)

    farm["mise_en_demeure"] = False
    if formal_notice(farm):
        farm["mise_en_demeure"] = True
    
    farm.pop("documentsHorsInspection", None)
    farm.pop("inspections", None)
    farm.pop("rubriques", None)
    
    return farm

farms = {k: alter_farm(v) for k, v in farms.items()}

## Export data

### Make dataframes

This allows easy csv creation

#### Farms

In [17]:
farm_records = list()

for farm_name, farm in farms.items():
    farm["name"] = farm_name
    farm_records.append(farm)

farms_df = pd.DataFrame.from_records(farm_records)
farms_df.head()

,adresse1,codePostal,codeInsee,commune,longitude,latitude,bovins,porcs,volailles,carriere,...,statutSeveso,ied,etatActivite,regimeVigueur,siret,mise_en_demeure,name,codeNaf,adresse2,adresse3
0,FERME DE FLASSY,89520,89331,SAINPUITS,3.257254,47.491203,False,True,False,False,...,Non Seveso,False,En exploitation avec titre,Enregistrement,38132669300011,False,GAEC DES BOURRYS,NaN,NaN,NaN
1,Pech Louis,81220,81266,ST PAUL CAP DE JOUX,1.988077,43.639220,False,True,False,False,...,Non Seveso,False,En fin d'exploitation,Autorisation,32841163200024,False,GAEC PORC D'ALGRES,NaN,NaN,NaN
2,LE HAMEL LELEGARD,50410,50393,PERCY-EN-NORMANDIE,-1.225119,48.913919,True,False,False,False,...,Non Seveso,False,En exploitation avec titre,Enregistrement,87966212000010,False,GAEC FERME H ET L,01,NaN,NaN
3,,50410,50393,PERCY EN NORMANDIE,-1.187543,48.950369,True,True,False,False,...,Non Seveso,False,En exploitation avec titre,Autorisation,42333815100016,False,BINOUDIERE (GAEC DE LA),NaN,La Binoudière,NaN
4,,50410,50393,PERCY EN NORMANDIE,-1.253302,48.918864,True,False,False,False,...,Non Seveso,False,En exploitation avec titre,Enregistrement,40832313700013,False,BOCAGE (GAEC DU),NaN,LE HAMEL AUX LOUVEAUX,NaN


#### Rubrics

In [18]:
rubrics_df = pd.DataFrame(rubrics.items(), columns=['id', 'nature'])
rubrics_df.head()

,id,nature
0,2102,Elevage de porcs de plus de 450 animaux-équiva...
1,3660,avec plus de 2 000 emplacements pour les porcs...
2,2101,"Bovins (élevage, vente, transit, etc)"
3,2111,Elevages de plus de 30 000 volailles
4,1532,Stockage bois déclaré


#### Stocks

In [19]:
stocks_df = pd.DataFrame.from_records(stocks)
stocks_df.quantite = pd.to_numeric(stocks_df.quantite)
stocks_df.head()

,raison_sociale,rubric_id,quantite,unite
0,GAEC DES BOURRYS,2102,1069.0,Animaux-Eq
1,GAEC PORC D'ALGRES,3660,860.0,
2,GAEC PORC D'ALGRES,2102,860.0,
3,GAEC FERME H ET L,2101,105.0,
4,GAEC FERME H ET L,2101,150.0,


### Write to file

In [20]:
farms_df.to_csv("farms.csv", index=False)
rubrics_df.to_csv("rubrics.csv", index=False)
stocks_df.to_csv("stocks.csv", index=False)